### Shape Detection Project : 
    1. Aim of this project:
        - Detecting shapes in a real world image using douglas-peucker algorithm

In [1]:
import numpy as np
import cv2
import math

In [2]:
img = int(input("""
      Choose one of the images below to detecting: 
      1-)normal, 2-)shade1, 3-)shade2, 4-)shade3, 5-)flash
      """))

In [3]:
#read , grayscale, blur
g_blur = 15
match img:
    case 1:
        img_normal = cv2.imread('normal.png') #adaptive mean
        img_normal_gray = cv2.cvtColor(img_normal, cv2.COLOR_BGR2GRAY)
        img_b = cv2.GaussianBlur(img_normal_gray, (g_blur, g_blur), 0)

    case 2:
        img_shade1 = cv2.imread('shade1.png') #adaptive gaussian
        img_shade_gray1 = cv2.cvtColor(img_shade1, cv2.COLOR_BGR2GRAY)
        img_b = cv2.GaussianBlur(img_shade_gray1, (g_blur, g_blur), 0)    

    case 3:
        img_shade2 = cv2.imread('shade2.png') #adaptive mean,gaussian
        img_shade_gray2 = cv2.cvtColor(img_shade2, cv2.COLOR_BGR2GRAY)
        img_b = cv2.GaussianBlur(img_shade_gray2, (g_blur, g_blur), 0)

    case 4:
        img_shade3 = cv2.imread('shade3.png') #adaptive gaussian 
        img_shade_gray3 = cv2.cvtColor(img_shade3, cv2.COLOR_BGR2GRAY)
        img_b = cv2.GaussianBlur(img_shade_gray3, (3, 3), 0)

    case 5:
        img_flash = cv2.imread('shapes_flas.jpg')
        img_flash_gray = cv2.cvtColor(img_flash, cv2.COLOR_BGR2GRAY)
        img_b = cv2.GaussianBlur(img_flash_gray, (g_blur, g_blur), 0)


SyntaxError: invalid syntax (1570469568.py, line 3)

In [99]:

# show the blurred image
cv2.imshow('shapes', img_b)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [100]:
#adaptive thresholding
# bunun sonucu siyah üzerine beyaz çizgili sınırlar
# shade 1 için işe yaramıyor
# block size 15: default 21 
# block size düşünce daha iyi sonuç elde ettim
match img:
    case 1:
        th = cv2.adaptiveThreshold(img_b, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)
     
    case 2:
        th = cv2.adaptiveThreshold(img_b, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV , 19, 10)
    
    case 3:
        th = cv2.adaptiveThreshold(img_b, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 10)
     
    case 4:
        th = cv2.adaptiveThreshold(img_b, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 23, 11)
    case 5:
        #flash için yaz
        pass
    

cv2.imshow('shapes', th)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [101]:
# first [144  49] second  [143  50]
def ssd(first, second):
    sum = 0 
    x_differ = abs(first[0]- second[0])
    y_differ = abs(first[1]- second[1])
    sum =  x_differ**2 + y_differ**2
    return sum

In [102]:
def simplyfy_list(corners):#[[[]],[[]]] ilk bulduğumu eledim diye köşeler kayıyor
    co = len(corners) #12
    ret_list = []
    for i in range(co): #0
        flag = True
        for j in range(i+1,co):
            ssd1 = ssd(corners[i][0],corners[j][0])
            if ssd1>100:
                flag = False
        if not flag:
            ret_list.append(corners[i][0])
        
    return ret_list

In [103]:
def angle_of_vectors(c1,c2):  # c1 =[236,233]
    a = c1[0]
    b = c1[1]
    c = c2[0]
    d = c2[1]
    dot_product = a*c + b*d
    # for three dimensional simply add dotProduct = a*c + b*d  + e*f 
    mod_of_vector1 = math.sqrt( a*a + b*b)*math.sqrt(c*c + d*d) 
    # for three dimensional simply add modOfVector = math.sqrt( a*a + b*b + e*e)*math.sqrt(c*c + d*d +f*f) 
    angle = dot_product/mod_of_vector1
    angle_degree = math.degrees(math.acos(angle))
    return angle_degree


In [104]:
#p1 = [ 81  50] ...
# vector = destination-source, p1 == source,p2 == destination 
def vector_of_points(p1,p2):
    x = p2[0]-p1[0]
    y = p2[1]-p1[1]
    return [x,y]

In [105]:
def douglas_peucker(points, epsilon):
    if len(points) <= 2:
        return points

    dmax = 0
    index = 0

    for i in range(1, len(points) - 1):
        d = perpendicular_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        part1 = douglas_peucker(points[:index + 1], epsilon)
        part2 = douglas_peucker(points[index:], epsilon)

        return np.vstack((part1[:-1], part2))
    else:
        return np.array([points[0], points[-1]])

def perpendicular_distance(point, line_start, line_end):
    if np.all(line_start == line_end):
        return np.linalg.norm(point - line_start)

    return np.abs(np.linalg.norm(np.cross(line_end - line_start, line_start - point))) / np.linalg.norm(line_end - line_start)

def is_square(contour):
    #contour (147,49),(79,51),(79,123),(153,120)
    is_square = False
    x=[]
    y=[]
    for i in range(len(contour)-1):
        first_corner = contour[i]
        x.append(abs(contour[i+1][0]-first_corner[0]))
        y.append(abs(contour[i+1][1]-first_corner[1]))
        
    ratio = max(x)/max(y)
    
    if 1.05> ratio>0.95:
        is_square = True
    return is_square


def is_line(contour):
    num = len(contour)
    if (num==2):
        return True
    else:
        for i in range(num-2): #contour[i] = [144  49]
            p1 = contour[i]
            p2 = contour[i+1]
            p3 = contour[i+2]
            v1 = vector_of_points(p1,p2)
            v2 = vector_of_points(p1,p3)
            ang = angle_of_vectors(v1,v2)
            if ang > 50:
                return True
            else:
                return False
        #slope gradientine bak

    
def get_polygon_type(contour):
    num_vertices = len(contour)
    if num_vertices == 3:
        return "Triangle"
    elif num_vertices == 4:
        if is_square(contour):
            return "Square"
        else:
            return "Rectangular"
       
    elif num_vertices == 5:
        return "Pentagon"
    elif num_vertices == 6:
        return "Hexagon"
    else:
        if(is_line(contour)):
            print("here")
            return "Line"
        else:
            return "Polygon or circle" 
                #circle minEnclosingCircle()

In [106]:
# Find contours
contours, hierarchy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#print(len(contours))

# Iterate through contours
for contour in contours:
    # Filter out small contours
    if cv2.contourArea(contour) < 100 :
        continue

    # Douglas-Peucker approximation with epsilon value (adjust as needed)
    epsilon = 9.0  # Adjust the epsilon value to control the approximation level bigger = more precise
    # epsilon 11 for shade3
    simplified_contour = douglas_peucker(contour[:, 0, :], epsilon)

    # Draw the simplified contour as a polygon
    simplified_contour = simplified_contour.reshape((-1, 1, 2)).astype(np.int32)
    
    #print(simplified_contour)
    match img:
        case 1:
            a = img_normal
        case 2:
            a =  img_shade1
        case 3:
            a =  img_shade2
        case 4:
            a =  img_shade3
        case 5:
            a =  img_flash
            
    # Get the type of polygon
    simplified_contour1 = simplyfy_list(simplified_contour)
    polygon_type = get_polygon_type(simplified_contour1)
    
    if not polygon_type=="Line":
       cv2.drawContours(a, [simplified_contour], 0, (0, 255, 255), 2) 
    
    #print(polygon_type)
    #print(simplified_contour1)
    # Calculate the centroid of the polygon
    M = cv2.moments(simplified_contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    if not polygon_type=="Line":
        # Write the polygon type as text near the centroid
        cv2.putText(a, polygon_type, (cX - 50, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
   


# Display the image with detected shapes
cv2.imshow('Shapes', a)
cv2.waitKey(0)
cv2.destroyAllWindows()


here
here
here
here
here
here


In [107]:

'''
yanliş açi değeri veriyor
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

p1 = [144,49]
p2 = [143,50]
p3 = [81,50]
v1 = vector_of_points(p1,p2)
v2 = vector_of_points(p1,p3)
ang = angle_between(v1,v2)
print(ang)
'''

'\nyanliş açi değeri veriyor\ndef unit_vector(vector):\n    """ Returns the unit vector of the vector.  """\n    return vector / np.linalg.norm(vector)\n\ndef angle_between(v1, v2):\n    """ Returns the angle in radians between vectors \'v1\' and \'v2\'::\n\n            >>> angle_between((1, 0, 0), (0, 1, 0))\n            1.5707963267948966\n            >>> angle_between((1, 0, 0), (1, 0, 0))\n            0.0\n            >>> angle_between((1, 0, 0), (-1, 0, 0))\n            3.141592653589793\n    """\n    v1_u = unit_vector(v1)\n    v2_u = unit_vector(v2)\n    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))\n\np1 = [144,49]\np2 = [143,50]\np3 = [81,50]\nv1 = vector_of_points(p1,p2)\nv2 = vector_of_points(p1,p3)\nang = angle_between(v1,v2)\nprint(ang)\n'